## Parking Controller via Online Optimization with Obstacles

###System dynamics:
(more info on https://nabinsharma.wordpress.com/2014/01/02/kinematics-of-a-robot-bicycle-model/)


Let  $x$ and $y$ the coordinate of the center of the rear-axel and $\phi$ the vehicle current orientation.
After a  straight segment of lenght $d_s$, the vehicle will be in 
\begin{align}
x' = x + d_s \sin (\phi),~\\
y' = y -d_s \cos (\phi),
\end{align}
assume not that it steers with and angle $\delta$, the turning radius is
\begin{align}
R = \frac{L}{\tan\delta}
\end{align}
and the turn angle is 
\begin{align}
\beta = \frac{d_\beta}{L} \tan\delta = \frac{d}{R}
\end{align}
where $d_\beta$ is the distance traveled during he turn

Coordinates of center of turn are,
\begin{align}
x_c &=& x' - R\sin\phi\\
y_c &=& y' + R\cos\phi.
\end{align}
New coordinated of the car after the turn are
\begin{align}
x^+ &=& x_c + R \sin (\phi + \beta)\\
y^+ &=& y_c - R \cos (\phi + \beta)\\
\phi^+ &=&(\phi + \beta).
\end{align}

Next the vectors are defined as follows.
State: $z = [x,y,\phi]$, Input: $u = [d_s, R, \beta]$, where $d_s$: the traveled straight  distance,
$R$ is the turning radius (after traveled $d_s$), $\beta$ is the traveled angle with turning radius $R$



###Optimization Algorithm

In [1]:
#Pkg.add("JuMP"); Pkg.add("Ipopt"); Pkg.add("Gadfly"); Pkg.add("Interact")
#Pkg.checkout("JuMP") # need latest master for matrices
using JuMP, Ipopt
using PyPlot
using PyCall

function solveMPC(car_size,n,m,T,z0,zT,zmin,zmax,umin,umax,obs,rep,side)
    l=car_size[1]
    dr=car_size[2]
    w=car_size[3]
    mpc = Model(solver=IpoptSolver(print_level=3))
    @defVar(mpc,  zmin[i] <= z[i=1:n,t=0:T] <= zmax[i])
    @defVar(mpc, -umax[i] <= u[i=1:m,t=0:T-1] <= umax[i])

    # Cost
    @setNLObjective(mpc, Min,sum{(u[2,t]*u[1,t])^2,t=0:T-1})
    
    # Link state and control across the horizon
    for t = 0:T-1        
        @addNLConstraint(mpc, z[1,t+1] == z[1,t] -u[1,t]*sin(z[3,t])+u[1,t]*sin(z[3,t]+u[2,t]))
        @addNLConstraint(mpc, z[2,t+1] == z[2,t] +u[1,t]*cos(z[3,t])-u[1,t]*cos(z[3,t]+u[2,t]))
        @addNLConstraint(mpc, z[3,t+1] == z[3,t] + u[2,t])
    end


    # Obstacle Avoidace constraints coded as disjunction
    if 0<1
        @defVar(mpc,  zmin[i] <= zs[i=1:n,t=0:rep*T] <= zmax[i])
        @addConstraint(mpc, zs[:,0] .== z0)
        t=0
        for k = 0:T-1        
            for j = 1:rep        
                t=t+1
                # Curved segment
                @addNLConstraint(mpc, zs[1,t] == zs[1,t-1] -u[1,k]*sin(zs[3,t-1])+u[1,k]*sin(zs[3,t-1]+1/rep*u[2,k]))
                @addNLConstraint(mpc, zs[2,t] == zs[2,t-1] +u[1,k]*cos(zs[3,t-1])-u[1,k]*cos(zs[3,t-1]+1/rep*u[2,k]))
                @addNLConstraint(mpc, zs[3,t] == zs[3,t-1] + 1/rep*u[2,k])
            end
        end
      
        # Obstacle Avoidace constraints coded as disjunction
        @defVar(mpc, λ[i=1:16,t=0:rep*T] >= 0)                     
        for t = 0:rep*T
            # Four corners of the car
            #A=[x+l*cos(psi)+w*sin(psi),y+l*sin(psi)-w*cos(psi)]
            #B=[x+w*sin(psi),y-w*cos(psi)]
            #C=[x+l*cos(psi)-w*sin(psi),y+l*sin(psi)+w*cos(psi)]
            #D=[x-w*sin(psi),y+w*cos(psi)]

            #To be parametrized later : parallel parking bay with two obstacles
            # x>=xl OR y-yt>=0 
            # x<=xr OR y-yt>=0 
            # Evaluated at A,B,C,D
            xl=obs[1]
            xr=obs[2]
            yt=obs[3]

            @addNLConstraint(mpc, λ[1,t]*(zs[1,t]+l*cos(zs[3,t])+w*sin(zs[3,t])-xl)+λ[2,t]*(zs[2,t]+l*sin(zs[3,t])-w*cos(zs[3,t])-yt) >=0)
            @addConstraint(mpc, λ[1,t]+λ[2,t] == 1)
            @addNLConstraint(mpc, λ[3,t]*(zs[1,t]-dr*cos(zs[3,t])+w*sin(zs[3,t])-xl)+λ[4,t]*(zs[2,t]-dr*sin(zs[3,t])-w*cos(zs[3,t])-yt) >=0)
            @addConstraint(mpc, λ[3,t]+λ[4,t] == 1)
            @addNLConstraint(mpc, λ[5,t]*(zs[1,t]+l*cos(zs[3,t])-w*sin(zs[3,t])-xl)+λ[6,t]*(zs[2,t]+l*sin(zs[3,t])+w*cos(zs[3,t])-yt) >=0)
            @addConstraint(mpc, λ[5,t]+λ[6,t] == 1)
            @addNLConstraint(mpc, λ[7,t]*(zs[1,t]-dr*cos(zs[3,t])-w*sin(zs[3,t])-xl)+λ[8,t]*(zs[2,t]-dr*sin(zs[3,t])+w*cos(zs[3,t])-yt) >=0)
            @addConstraint(mpc, λ[7,t]+λ[8,t] == 1)

            @addNLConstraint(mpc, λ[9,t]*(xr-(zs[1,t]+l*cos(zs[3,t])+w*sin(zs[3,t])))+λ[10,t]*(zs[2,t]+l*sin(zs[3,t])-w*cos(zs[3,t])-yt) >=0)
            @addConstraint(mpc, λ[9,t]+λ[10,t] == 1)     
            @addNLConstraint(mpc, λ[11,t]*(-(zs[1,t]-dr*cos(zs[3,t])+w*sin(zs[3,t]))+xr)+λ[12,t]*(zs[2,t]-dr*sin(zs[3,t])-w*cos(zs[3,t])-yt) >=0)
            @addConstraint(mpc, λ[11,t]+λ[12,t] == 1)
            @addNLConstraint(mpc, λ[13,t]*(-(zs[1,t]+l*cos(zs[3,t])-w*sin(zs[3,t]))+xr)+λ[14,t]*(zs[2,t]+l*sin(zs[3,t])+w*cos(zs[3,t])-yt) >=0)
            @addConstraint(mpc, λ[13,t]+λ[14,t] == 1)
            @addNLConstraint(mpc, λ[15,t]*(-(zs[1,t]-dr*cos(zs[3,t])-w*sin(zs[3,t]))+xr)+λ[16,t]*(zs[2,t]-dr*sin(zs[3,t])+w*cos(zs[3,t])-yt) >=0)
            @addConstraint(mpc, λ[15,t]+λ[16,t] == 1)
            
            
            @addNLConstraint(mpc, zs[2,t]+l*sin(zs[3,t])-w*cos(zs[3,t]) >=zmin[2])
            @addNLConstraint(mpc, zs[2,t]-dr*sin(zs[3,t])-w*cos(zs[3,t])>=zmin[2])
            @addNLConstraint(mpc, zs[2,t]+l*sin(zs[3,t])+w*cos(zs[3,t]) >=zmin[2])
            @addNLConstraint(mpc, zs[2,t]-dr*sin(zs[3,t])+w*cos(zs[3,t])>=zmin[2])
            @addNLConstraint(mpc, zs[2,t]+l*sin(zs[3,t])-w*cos(zs[3,t]) <=zmax[2])
            @addNLConstraint(mpc, zs[2,t]-dr*sin(zs[3,t])-w*cos(zs[3,t])<=zmax[2])
            @addNLConstraint(mpc, zs[2,t]+l*sin(zs[3,t])+w*cos(zs[3,t]) <=zmax[2])
            @addNLConstraint(mpc, zs[2,t]-dr*sin(zs[3,t])+w*cos(zs[3,t])<=zmax[2])
 

        end
    end
    
    for t = 0:T-1 
        if side[t+1]=='1'
           #Postitive Curvature Radius
           @addConstraint(mpc, u[1,t] >= umin[1])
           #print("added +,")
        else
            #Negative Curvature Radius
           @addConstraint(mpc, u[1,t] <= -umin[1])
           #print("added -,")
        end
    end

    # Initial conditions
    @addConstraint(mpc, z[:,0] .== z0)
    # Final state
    @addConstraint(mpc, z[:,T] .== zT)   
    # Solve the NLP
    status= solve(mpc)
    return getValue(u), getValue(z), getObjectiveValue(mpc), status
end

INFO: Loading help data...


solveMPC (generic function with 1 method)

### Run Simulations

In [7]:
#Car Parameters
n=3
m=2
lr  = 1.7
dr = 1.2
lf = 1.1
df = 1.0
width=1
delta_max=25*pi/180
L  = lr+lf+df
car_size=[L,dr,width]

# rep= number of sampling points for each segment -- for obstacle checking
#Increase resolution in space if obstacle are not satisfied
rep=5

# Constraints
zmin=[-20; 0;-2*pi]
umax=[2000;pi]
umin=[(lf+lr)/tan(delta_max);-umax[2]]


# Initial conditions, terminal condition and parkyng bay size
#  EXAMPLE A - Vertical Parking ~1 min
zmax=[20;20;2*pi]
z0 = [-10;10;0]  
zT = [0;1+dr;pi/2]  
obs=[-2;2;L+1]
Tmpc = 2 # MPC horizon
rep=5

# EXAMPLE B- Parallel Parking 
#z0 = [8.5;6;0]  
#zT = [1.3;1.5;pi]
#obs=[-3.5;3.5;3]
#zmax=[15;10;2*pi]
#zmin=[-15;0;-2*pi]
#Tmpc = 6 # MPC horizon Solution time ~40 min, index optimal =37
#rep=5


bestcost=100000000
usol= zeros(n,Tmpc)
zsol = zeros(m,Tmpc)
# MPC
tic()
for j=0:(2^Tmpc)-1
#for j=0:0
    # check all positive or negative combinatiosn of curvature radius
    side=bin(j,Tmpc)
    #side=bin(37,Tmpc)
    #print(side)
    #solve Optimization problem for a given gurvature sign sequence
    u_vec, z_vec, cost, status= solveMPC(car_size,n,m,Tmpc,z0,zT,zmin,zmax,umin,umax,obs,rep,side)
    print("NPSOL: ",j,"--")
    if status==:Optimal
        if cost<bestcost
            bestcost=cost
            print("new best index:",j)
            usol=u_vec
            zsol=z_vec
        end
    end
end
print("best_cost: ",bestcost)
#print(zsol)
print(usol)
toc()
#37


added -,added -,NPSOL: 0--new best index:0added -,added +,

NPSOL: 1--added +,added -,NPSOL: 2--added +,added +,

NPSOL: 3--best_cost: 421.62547691832117u: 2 dimensions:
[1,:]
  [1,0] = -78.23755404140363
  [1,1] = -6.004619317465848
[2,:]
  [2,0] = -0.22342373993702536
  [2,1] = 1.794220066731922
elapsed time: 34.697303776 seconds


34.697303776

Traceback (most recent call last):
  File "C:\Users\Francesco\Anaconda3\lib\site-packages\matplotlib\backend_bases.py", line 1290, in _on_timer
    ret = func(*args, **kwargs)
  File "C:\Users\Francesco\Anaconda3\lib\site-packages\matplotlib\animation.py", line 925, in _step
    still_going = Animation._step(self, *args)
  File "C:\Users\Francesco\Anaconda3\lib\site-packages\matplotlib\animation.py", line 784, in _step
    self._draw_next_frame(framedata, self._blit)
  File "C:\Users\Francesco\Anaconda3\lib\site-packages\matplotlib\animation.py", line 803, in _draw_next_frame
    self._draw_frame(framedata)
  File "C:\Users\Francesco\Anaconda3\lib\site-packages\matplotlib\animation.py", line 1106, in _draw_frame
    self._drawn_artists = self._func(framedata, *self._args)
IndexError: Julia exception: BoundsError()
Traceback (most recent call last):
  File "C:\Users\Francesco\Anaconda3\lib\site-packages\matplotlib\backend_bases.py", line 1290, in _on_timer
    ret = func(*args, **kwargs

In [3]:
function simulate_car_MPC(Tmpc,z0,u,rep,L)
    Tsim=rep*Tmpc
    us = zeros(1,Tsim)
    zs = zeros(3,Tsim+1)
    t=1
    zs[:,1] = z0[:]
    for k = 0:Tmpc-1        
             for l = 1:rep       
                t=t+1
                zs[1,t] = zs[1,t-1] -u[1,k]*sin(zs[3,t-1])+u[1,k]*sin(zs[3,t-1]+1/rep*u[2,k])
                zs[2,t] = zs[2,t-1] +u[1,k]*cos(zs[3,t-1])-u[1,k]*cos(zs[3,t-1]+1/rep*u[2,k])
                zs[3,t] = zs[3,t-1] + 1/rep*u[2,k]
            us[t-1]= atan(L/u[1,k])
            end
    end
    
    return  us, zs
end


simulate_car_MPC2 (generic function with 1 method)

###Next cell plots the parking manouver

In [8]:
@pyimport matplotlib.pyplot as plt

# Generate vectors for plots:
#print(usol)
#print(zsol)
rep=10
Tsim=rep*Tmpc
u_vec,z_vec=simulate_car_MPC(Tmpc,z0,usol,rep,L)
#print(z_vec)

#Find Axis Limits
xmin = minimum([z_vec[1,:]])
xmax = maximum([z_vec[1,:]])
xmin, xmax = xmin - 0.1(xmax-xmin), xmax+ 0.1*(xmax-xmin)
ymin = minimum([z_vec[2,:]])
ymax = maximum([z_vec[2,:]])
ymin, ymax = ymin - 0.1(ymax-ymin), ymax+ 0.1*(ymax-ymin)

function plot_car(x,y,psi,beta,car_size,pt)
    l=car_size[1]
    dr=car_size[2]
    w=car_size[3]
    A=[x+l*cos(psi)+w*sin(psi),y+l*sin(psi)-w*cos(psi)]
    B=[x-dr*cos(psi)+w*sin(psi),y-dr*sin(psi)-w*cos(psi)]
    C=[x+l*cos(psi)-w*sin(psi),y+l*sin(psi)+w*cos(psi)]
    D=[x-dr*cos(psi)-w*sin(psi),y-dr*sin(psi)+w*cos(psi)]
    #wheels
    lw=0.4
    E=[A[1]+lw*cos(psi+beta),A[2]+lw*sin(psi+beta)]
    F=[C[1]+lw*cos(psi+beta),C[2]+lw*sin(psi+beta)]
    
    pt.plot([B[1] ,A[1]],[B[2], A[2]],"bo-")
    pt.plot([C[1] ,D[1]],[C[2], D[2]],"bo-")
    pt.plot([C[1] ,A[1]],[C[2], A[2]],"bo-")
    pt.plot([B[1] ,D[1]],[B[2], D[2]],"bo-")
    pt.plot([A[1] ,E[1]],[A[2], E[2]],"go-")
    pt.plot([C[1] ,F[1]],[C[2], F[2]],"go-")
end


#Construct Figure and Plot Data
fig = figure()
ax = plt.axes()
ax = plt.axes(xlim = (-20,20),ylim=(-2,20))
#ax = plt.axes(xlim = (xmin,xmax),ylim=(ymin,ymax))
plt.plot(transpose(z_vec[1,:]),transpose(z_vec[2,:]), "r-")

#Plot car and Obstacles
for i=1:1:Tsim
    plot_car(z_vec[1,i],z_vec[2,i],z_vec[3,i],u_vec[1,i],car_size,plt)
end
plot_car(z_vec[1,[end]],z_vec[2,[end]],z_vec[3,[end]],u_vec[1,[end]],car_size,plt)

plt.plot([obs[1] ,obs[1]],[0, obs[3]],"r-")
plt.plot([obs[2] ,obs[2]],[0, obs[3]],"r-")
plt.plot([-20 ,obs[1]],[obs[3], obs[3]],"r-")
plt.plot([obs[2] ,20],[obs[3], obs[3]],"r-")
plt.plot([-20 ,20],[zmax[2], zmax[2]],"r-")
plt.show()


#Plot start and end points
#ax[:plot](z_vec[1,1],z_vec[2,1], "ro")
#ax[:plot](z_vec[1,end],z_vec[2,end], "rs")

u: 2 dimensions:
[1,:]
  [1,0] = -78.23755404140363
  [1,1] = -6.004619317465848
[2,:]
  [2,0] = -0.22342373993702536
  [2,1] = 1.794220066731922


### Next makes a video of the manouver

In [9]:
#Pkg.add("PyPlot")
#Pkg.add("PyCall")
using PyPlot
using PyCall
@pyimport matplotlib.animation as anim
pygui(true)
Tsim=rep*Tmpc

# First set up the figure, the axis, and the plot element we want to animate
fig = figure()
#ax = plt.axes(xlim = (xmin,xmax),ylim=(ymin,ymax))
#ax = plt.axes(xlim=(-10, 5), ylim=(-2, 10))
ax = plt.axes(xlim=(-15, 15), ylim=(-1, 20))
#ax = plt.axes(xlim=(-5, 10), ylim=(-1, 14))
#ax2 = plt.axes()
global line1 = ax[:plot]([], [], "ro-")[1]
global line2 = ax[:plot]([], [], "ro-")[1]
global line3 = ax[:plot]([], [], "ro-")[1]
global line4 = ax[:plot]([], [], "ro-")[1]
global line5 = ax[:plot]([], [], "g-")[1]
global line6 = ax[:plot]([], [], "g-")[1]
global line7 = ax[:plot]([], [], "b--")[1]
ax[:plot]([obs[1] ,obs[1]],[0, obs[3]],"r-")
ax[:plot]([obs[2] ,obs[2]],[0, obs[3]],"r-")
ax[:plot]([-20 ,obs[1]],[obs[3], obs[3]],"r-")
ax[:plot]([obs[2] ,20],[obs[3], obs[3]],"r-")
ax[:plot]([-20 ,20],[zmax[2], zmax[2]],"r-")
ax[:plot]([-20 ,20],[0, 0],"r-")

function points_car(x,y,psi,beta,car_size)
    l=car_size[1]
    dr=car_size[2]
    w=car_size[3]

    A=[x+l*cos(psi)+w*sin(psi),y+l*sin(psi)-w*cos(psi)]
    B=[x-dr*cos(psi)+w*sin(psi),y-dr*sin(psi)-w*cos(psi)]
    C=[x+l*cos(psi)-w*sin(psi),y+l*sin(psi)+w*cos(psi)]
    D=[x-dr*cos(psi)-w*sin(psi),y-dr*sin(psi)+w*cos(psi)]
    #wheels
    lw=1
    E=[A[1]+lw/2*cos(psi+beta),A[2]+lw/2*sin(psi+beta)]
    F=[C[1]+lw/2*cos(psi+beta),C[2]+lw/2*sin(psi+beta)]
    G=[A[1]-lw/2*cos(psi+beta),A[2]-lw/2*sin(psi+beta)]
    H=[C[1]-lw/2*cos(psi+beta),C[2]-lw/2*sin(psi+beta)]
    return A,B,C,D,E,F,G,H
end
 

# initialization function: plot the background of each frame
function init()
    global line1
    global line2
    global line3
    global line4
    global line5
    global line6
    global line7
    line1[:set_data]([], [])
    line2[:set_data]([], [])
    line3[:set_data]([], [])
    line4[:set_data]([], [])
    line5[:set_data]([], [])
    line6[:set_data]([], [])
    line7[:set_data]([], [])
    return (line1,line2,line3,line4,line5,line6,line7,None)
end

# animation function.  This is called sequentially
function animate(i)
    k=i+1
    global line1
    global line2
    global line3
    global line4
    global line5
    global line6
    global line7
    if k<=Tsim
        A,B,C,D,E,F,G,H=points_car(z_vec[1,k],z_vec[2,k],z_vec[3,k],u_vec[k],car_size)
        line7[:set_data](z_vec[1,1:k],z_vec[2,1:k])
    else
        A,B,C,D,E,F,G,H=points_car(z_vec[1,end],z_vec[2,end],z_vec[3,end],u_vec[end],car_size)
        line7[:set_data](z_vec[1,1:end],z_vec[2,1:end])
    end
    line1[:set_data]([B[1] ,A[1]],[B[2], A[2]])
    line2[:set_data]([C[1] ,D[1]],[C[2], D[2]])
    line3[:set_data]([C[1] ,A[1]],[C[2], A[2]])
    line4[:set_data]([B[1] ,D[1]],[B[2], D[2]])
    line5[:set_data]([G[1] ,E[1]],[G[2], E[2]])
    line6[:set_data]([H[1] ,F[1]],[H[2], F[2]])
    return (line1,line2,line3,line4,line5,line6,line7,None)
end


# call the animator.  blit=True means only re-draw the parts that have changed.
myanim = anim.FuncAnimation(fig, animate, init_func=init, frames=Tsim+20,  interval=100,)
#myanim[:save]("p", writer="ffmpeg",  extra_args=["-vcodec", "libx264"])
#myanim[:save]("parking2.mp4", extra_args=["-vcodec", "libx264", "-pix_fmt", "yuv420p"])
myanim[:save]("./parking5.mp4", extra_args=["-vcodec", "libx264", "-pix_fmt", "yuv420p"])